## Demo for ["Exact and optimal quadratization of nonlinear finite-dimensional non-autonomous dynamical systems"](https://arxiv.org/abs/2303.10285)

This notebook contains several examples of usage of QBee following Sections 5-7 of the paper

In [1]:
# Loading packages
import sympy as sp
from qbee import *

## Introductory examples (Section 5)

### Basic usage of QBee (Section 5.1)

First we show how one can use QBee to quadratize an autonomous finite-dimensional ODE system such as
$$
\begin{cases}
\dot{x}_1 = x_1^2 + x_2^2,\\
\dot{x}_2 = x_1 + x_2
\end{cases}
$$

In [2]:
x1, x2 = functions("x1, x2")
system = [ # pairs of the form (x, x')
    (x1, x1**3 + x2**2),
    (x2, x1 + x2)
]
quadratize(system).print()

Introduced variables:
w0 = x1**2
w1 = x2**2

x1' = w0*x1 + w1
x2' = x1 + x2
w0' = 2*w0**2 + 2*w1*x1
w1' = 2*w1 + 2*x1*x2


We have thus obtained a monomial quadratization of order two which is guaranteed to be optimal among the monomial quadratizations.

### Polynomial ODEs with inputs (Section 5.2)

Now we proceed to the new functionality such as systems with inputs. Consider a scalar ODE from Example 5.2
$$
\dot{x} = x^2u
$$
It can be quadratized using QBee as follows.

In [3]:
x, u = functions("x, u")
system = [ # pairs of the form (x, x')
    (x, x**2 * u)
]
quadratize(system).print() # input-free=False by default

Introduced variables:
w0 = u*x

x' = w0*x
w0' = u'*x + w0**2


We observe that the resulting quadratic system involves the derivative of the input, that is, $\dot{u}$. As explained in Example 3.8 of the paper, in this particular case, this was unavoidable. However, in many cases one can find a quadratization which does not involve the derivatives of the inputs (we call such quadratizations *input-free*). Search for input-free quadratizations can be forced by setting the `input_free` keyword argument to be `True`. 

As an example, consider a system
$$
\begin{cases}
  \dot{x}_1 = x_1 + x_1u,\\
  \dot{x}_2 = x_1^2 u.
\end{cases}
$$
The following code can be used to find an input-free quadratization for it.

In [4]:
x1, x2, u = functions("x1, x2, u")
system = [ # pairs of the form (x, x')
    (x1, x1 + x1 * u),
    (x2, x1**2 * u)
]
quadratize(system, input_free=True).print()

Introduced variables:
w0 = x1**2

x1' = u*x1 + x1
x2' = u*w0
w0' = 2*u*w0 + 2*w0


And we have obtained an optimal monomial input-free quadratization of order one.

### Dimension-agnostic quadratization (Section 5.3)

We will now show how QBee can be applied to systems of variable dimension occurring, for example, as discretizations of PDEs. We will consider a PDE coming from traffic modeling (see Example 4.1 for further details):
$$
\frac{\partial \rho}{\partial t}(t, \xi) = \rho(t, \xi) + \rho^2(t, \xi) \frac{\partial \rho(t, \xi)}{\partial \xi}, \qquad \rho(t, 0) = 0, \quad \rho(t, 1) = 1.
$$
For any integer $n$, we can discretize the spatial domain uniformly $x_{i}^{[n]}(t) = \rho(t, i/(n + 1))$.
Then the vector $\mathbf{x}^{[n]}(t) = [x_{1}^{[n]}(t), \ldots, x_{n}^{[n]}(t)]^\top$ satisfies an ODE system of dimension $n$
$$
\dot{\mathbf{x}}^{[n]} = \mathbf{x}^{[n]} + (\mathbf{x}^{[n]})^2 \odot (\mathbf{D}\mathbf{x}^{[n]}),
$$
where $\mathbf{D}$ is the matrix for taking the finite difference.

QBee allows to describe (in finite terms) a set of new variables which will yield a quadratization for any $n$ and any matrix $\mathbf{D}$ (we call this *dimension-agnostic* quadratization). The system above can be encoded as follows (see Example 5.4).

In [5]:
x, Dx = functions("x Dx")
system = [
    (x, x + x**2 * Dx)
]
quadratize_dimension_agnostic(system);

Every ODE system in the family can be quadratized by adding the following variables
	* For each i, we take variables
	x_i**2
	* For every i and j such that the variables from the j-th block (that is, x_j) appear in the right-hand sides of the i-th block (that is, in x_i'), we take
	x_i*x_j


And we see that the result of the computation is a recipy how to construct a quadratization depending on $n$ and the positions of the nonzero entries in $\mathbf{D}$.

### Polynomialization with QBee (Section 5.4)

QBee can also take as input a systems with nonpolynomial right-hand side and lift it to a polynomial one. This can be used as a preprocessing for quadratization but also as a feature on its own. Consider, for example, a scalar nonpolynomial ODE from Example 5.5 
$$
\dot{x} = e^{-x} + e^{-2x}. 
$$
This ODE can be lifted to a polynomial system using QBee as follows.

In [6]:
x = functions("x")
system = [(x, sp.exp(-x) + sp.exp(-2 * x))] # list of pairs (x, x')
print(polynomialize(system))

x' = w_0**2 + w_0
w_0' = -w_0*(w_0**2 + w_0)


Note that we got a cubic system which can be further quadratized. In fact, this can be done by a single call.

In [7]:
polynomialize_and_quadratize(system)

w_0 = exp(-x)
w_1 = w_0**2

x' = w_0 + w_1
w_0' = -w_0*w_1 - w_1
w_1' = -2*w_0*w_1 - 2*w_1**2

## Examples from the literature

### Tubular reactor model (Section 6.1)

We consider two versions of a non-adiabatic tubular reactor model with a single reaction: with polynomial and exponential reaction terms.
In both cases, the ODE system of interest will be a result of spatial discretization of a PDE system with two vectors of states: species concentration $\boldsymbol{\psi}(t)$ and temperature $\boldsymbol{\theta}(t)$.

The *polynomial* version (Section 6.1.1) will be a $2n$ dimensional system of ODEs with scalar input $u(t)$ and several scalar or vector parameters:
$$
\begin{cases}
  \boldsymbol{\dot{\psi}}  = \mathbf{A}_\psi \boldsymbol{\psi} + \mathbf{b}_\psi - \mathcal{D}\boldsymbol{\psi} \odot (\mathbf{c}_0 + \mathbf{c}_1 \odot\boldsymbol{\theta} + \mathbf{c}_2 \odot\boldsymbol{\theta}^2 + \mathbf{c}_3 \odot\boldsymbol{\theta}^3), \\
  \boldsymbol{\dot{\theta}} = \mathbf{A}_\theta \boldsymbol{\theta} + \mathbf{b}_\theta + \mathbf{b} u + \mathcal{B} \mathcal{D} \boldsymbol{\psi} \odot (\mathbf{c}_0 + \mathbf{c}_1 \odot\boldsymbol{\theta} + \mathbf{c}_2 \odot\boldsymbol{\theta}^2 + \mathbf{c}_3 \odot\boldsymbol{\theta}^3),
\end{cases}
$$
We can search for a dimension-agnostic quadratization of this model using QBee as follows.

In [8]:
# Dpsi and Dtheta below refer to the matrix-vector products A_psi * psi and A_theta * theta
# the letter D in from of the variable name should be used so that QBee matches the vector and
# its transforms
psi, theta, w1, Dpsi, Dtheta = functions("psi theta w1 Dpsi Dtheta")
u = functions("u")
B, D, gamma, bpsi, btheta, b = parameters("B D gamma bpsi btheta b")
c0, c1, c2, c3 = parameters("c0 c1 c2 c3")

poly = c0 + c1 * theta + c2 * theta**2 + c3 * theta**3
system = [
    (psi, Dpsi + bpsi - D * psi * poly),
    (theta, Dtheta + btheta + b * u + B * D * psi * poly),
]
quadratize_dimension_agnostic(system);

Found inputs u, they will be considered space-independent


Every ODE system in the family can be quadratized by adding the following variables
	* For each i, we take variables
	theta_i**2, psi_i*theta_i, psi_i*theta_i**2, theta_i**3


This means that the system can be always quadratized (independently of the shapes of $\mathbf{A}_\psi$ and $\mathbf{A}_\theta$) with $4n$ extra variables $\boldsymbol{\theta}^2,\; \boldsymbol{\psi}\odot \boldsymbol{\theta},\; \boldsymbol{\psi}\odot\boldsymbol{\theta}^2,\; \boldsymbol{\theta}^3$.

The *exponential* version of the model uses the same states but involves exponential function.
$$
\begin{cases}
\boldsymbol{\dot{\psi}} = \mathbf{A}_{\psi} \boldsymbol{\psi} + \mathbf{b}_\psi - \mathcal{D} \boldsymbol{\psi} \odot e^{\gamma - \gamma / \boldsymbol{\theta}},\\
\boldsymbol{\dot{\theta}} = \mathbf{A}_{\theta} \boldsymbol{\theta} + \mathbf{b}_\theta + \mathbf{b} u(t) + \mathcal{B} \mathcal{D} \boldsymbol{\psi} \odot e^{\gamma - \gamma / \bm{\theta}}
\end{cases}
$$
QBee cannot find dimension-agnostic quadratizations with arbitrary non-polynomial right-hand side but can handle the case of Laurent monomials (i.e. negative powers). In order to reduce to this case, we manually introduce $\mathbf{w}_1 = e^{\gamma - \gamma / \boldsymbol{\theta}}$ and obtain
$$
\begin{cases}
\boldsymbol{\dot{\psi}}  = \mathbf{A}_{\psi} \boldsymbol{\psi} + \mathbf{b}_\psi - \mathcal{D} \boldsymbol{\psi} \odot \mathbf{w}_1,\\
\boldsymbol{\dot{\theta}} & = \mathbf{A}_{\theta} \boldsymbol{\theta} + \mathbf{b}_\theta + \mathbf{b} u(t) + \mathcal{B} \mathcal{D} \boldsymbol{\psi} \odot \mathbf{w}_1,\\
      \dot{\mathbf{w}}_1 & = \gamma\boldsymbol{\dot{\theta}} \odot \frac{1}{\boldsymbol{\theta}^2} \odot \boldsymbol{w}_1.
\end{cases}
$$
Now we can, similarly to the polynomial version, obtain a dimension-agnostic quadratization using QBee

In [9]:
psi, theta, w1, Dpsi, Dtheta = functions("psi theta w1 Dpsi Dtheta")
u = functions("u")

B, D, gamma, bpsi, btheta, b = parameters("B D gamma bpsi btheta b")

theta_diff = Dtheta + btheta + b * u + B * D * psi * w1
system = [
    (psi, Dpsi + bpsi - D * psi * w1),
    (theta, theta_diff),
    (w1, gamma * w1 / theta**2 * theta_diff)
]
quadratize_dimension_agnostic(system);

Found inputs u, they will be considered space-independent


Nodes processed: 383 nodes [00:04, 83.85 nodes/s, Current best order = 36, To return current best press Ctrl+C] 

The quadratization algorithm has been interrupted. Returning the current best...


Nodes processed: 248 nodes [00:03, 77.36 nodes/s, Current best order = 36, To return current best press Ctrl+C]

The quadratization algorithm has been interrupted. Returning the current best...


Every ODE system in the family can be quadratized by adding the following variables
	* For each i, we take variables
	1/theta_i, theta_i**(-2), u/theta_i, u/theta_i**2, psi_i*w1_i/theta_i**2, psi_i*w1_i/theta_i
	* For every i and j such that the variables from the j-th block (that is, w1_j, theta_j, psi_j) appear in the right-hand sides of the i-th block (that is, in w1_i', theta_i', psi_i'), we take
	theta_j/theta_i, psi_j/theta_i, theta_j/theta_i**2, psi_j/theta_i**2


The full computation for this example will take some time (~20 minutes) but already after one minute it will display `Current best order = 36` meaning that some quadratization has been found (the exact number 36 can be ignored, it originates from the depths of the algorithm) and the algorithm is trying to refine it. One can interrupt the computation, and the current best option will be displayed (as above). Interestingly, it turns out that this is indeed the best quadratization to be found. Compared to the polynomial version, the shapes of the matrices matter now.

### Species’ reaction model for combustion (Section 6.2)

We consider the rocket engine combustion model which is a non-polynomial non-autonomous ODE system in four variables $x_1, x_2, x_3, x_4$ denoting the concentrations of different species and with one external input $u(t)$.
$$
\begin{cases}
    \dot{x}_1 = -A \exp{\Big(-\frac{E}{R u(t)}\Big)} x_1^{0.2} x_2^{1.3}\\
    \dot{x}_2 = 2 \dot{x}_1\\
    \dot{x}_3 = -\dot{x}_1 \\
    \dot{x}_4 = -2 \dot{x}_1.
\end{cases}
$$
The system can be polynomialized and then quadratized using QBee as follows.

In [10]:
x1, x2, x3, x4 = functions("x1, x2, x3, x4")
u = functions("u")
A, E, R = parameters("A, E, R")
x1_diff = -A * sp.exp(-E / (R * u)) * x1**0.2 * x2**1.3

system = [
    (x1, x1_diff),
    (x2, 2 * x1_diff),
    (x3, -x1_diff),
    (x4, -2 * x1_diff)
]
# we allow the input order go up to two because u' will already appear at the polynomialization step
polynomialize_and_quadratize(system, input_der_orders={u: 2})

w_0 = x2**1.3
w_1 = exp(-E/(R*u))
w_2 = x1**0.2
w_3 = w_0*w_1
w_4 = u'/u**2
w_5 = u**(-2)
w_6 = u'/u
w_7 = 1/u
w_8 = w_0*w_1*w_2/x1
w_9 = w_0*w_1*w_2/x2

x1' = -A*w_2*w_3
x2' = -2*A*w_2*w_3
x3' = A*w_2*w_3
x4' = 2*A*w_2*w_3
w_0' = -13*A*w_0*w_9/5
w_1' = E*w_1*w_4/R
w_2' = -A*w_2*w_8/5
w_3' = -13*A*w_3*w_9/5 + E*w_3*w_4/R
w_4' = u''*w_5 - 2*w_4*w_6
w_5' = -2*w_5*w_6
w_6' = u''*w_7 - w_6**2
w_7' = -w_6*w_7
w_8' = 4*A*w_8**2/5 - 13*A*w_8*w_9/5 + E*w_4*w_8/R
w_9' = -A*w_8*w_9/5 - 3*A*w_9**2/5 + E*w_4*w_9/R

As a result, we get a quadratization of order 10 which involves the second derivative of the input (it is possible to prove that it is impossible to have a lower order derivative of the input).

### Solar wind modeling (Section 7)

The solar wind model we consider is a result of semi-discretization of a PDE and it is an $n$-dimensional ODE system in a vector of unknown functions $\mathbf{v}(r)$ in the $r$ variable:
$$
\frac{\text{d}\mathbf{v}(r)}{\text{d}r} = \mathbf{D}\ln \left[\mathbf{v}(r)\right] - \frac{\xi}{\Omega_\text{rot}(\hat{\theta})}\mathbf{D} \mathbf{v}(r),
$$
where $\mathbf{D}$ is the matrix encoding the finite difference scheme. In order to use QBee to search for a dimension-agnostic quadratization, we eliminate the logarithmic term by introducing $\mathbf{w}_0 := \ln (\mathbf{v})$. We also set $C_1 := \frac{\xi}{\Omega_{\text{rot}}(\hat{\theta})}$ to simplify the notation. We therefore obtain a $2n$-dimensional ODE model
$$
\begin{cases}
\frac{\text{d} \mathbf{v}}{\text{d}r} = \mathbf{D} \mathbf{w}_0 - C_1 \mathbf{D} \mathbf{v},\\
  \frac{\text{d} \mathbf{w}_0}{\text{d}r} = \frac{1}{\mathbf{v}}\mathbf{D} \mathbf{w}_0 - \frac{C_1}{\mathbf{v}}\mathbf{D} \mathbf{v}.
\end{cases}
$$
Now we can use QBee to find a dimension-agnostic quadratization.

In [11]:
u, v, Du, Dv = functions("u v Du Dv")
C1 = parameters("C1")
system = [
    (v, Du + C1 * Dv),
    (u, Du / v + C1 * Dv / v)
]
quadratize_dimension_agnostic(system);

Every ODE system in the family can be quadratized by adding the following variables
	* For each i, we take variables
	1/v_i, u_i/v_i
	* For every i and j such that the variables from the j-th block (that is, v_j, u_j) appear in the right-hand sides of the i-th block (that is, in v_i', u_i'), we take
	u_j/v_i, v_j/v_i


As explained in the paper, for the particular matrix $\mathbf{D}$ of interest, one can reduce the produced quadratization further by hand.